In [1]:
import pandas as pd
datafile = "titanic.csv"
data = pd.read_csv(datafile)

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
data['Gender']=data['Sex'].apply(lambda a: 1 if a=='male' else 0)

data=data.drop(['PassengerId','Name','Cabin','Embarked','Ticket','Sex'],axis=1)

data=data.dropna()

data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1


In [3]:
from sklearn.cross_validation import train_test_split
train, test = train_test_split(data, test_size = 0.2)

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
287,0,3,22.0,0,0,7.8958,1
628,0,3,26.0,0,0,7.8958,1
838,1,3,32.0,0,0,56.4958,1
886,0,2,27.0,0,0,13.0000,1
518,1,2,36.0,1,0,26.0000,0


In [5]:
test.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
173,0,3,21.0,0,0,7.9250,1
319,1,1,40.0,1,1,134.5000,0
632,1,1,32.0,0,0,30.5000,1
546,1,2,19.0,1,0,26.0000,0
710,1,1,24.0,0,0,49.5042,0


In [16]:
from sklearn.tree import DecisionTreeClassifier
#clf=DecisionTreeClassifier(max_leaf_nodes=15)
clf=DecisionTreeClassifier()
clf=clf.fit(train.drop(['Survived'],axis=1).values,train["Survived"].values)

In [17]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [18]:
clf.feature_importances_

array([ 0.10907166,  0.26547396,  0.05165015,  0.00920891,  0.26343065,
        0.30116467])

In [19]:
from sklearn import tree
with open("titanic.dot","w") as f:
    f = tree.export_graphviz(clf,
                            feature_names=train.drop(['Survived'],axis=1).columns,out_file=f)

In [20]:
predictions = clf.predict(test.drop(['Survived'],axis=1).values)

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(test["Survived"], predictions)

0.79720279720279719

In [22]:
from sklearn.ensemble import RandomForestClassifier

X_train=train.drop(['Survived'],axis=1).values
y_train=train["Survived"].values
X_test=test.drop(['Survived'],axis=1).values
y_test=test["Survived"]

clf = RandomForestClassifier(n_estimators=1000,max_leaf_nodes=15)
clf=clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.82517482517482521

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf=clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.85314685314685312

In [24]:
from sklearn import svm
clf = svm.SVC()

clf=clf.fit(X_train, y_train)  
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.67132867132867136

In [25]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2")

clf=clf.fit(X_train, y_train)  
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.75524475524475521

In [26]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

clf=clf.fit(X_train, y_train)  
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.76923076923076927

In [23]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)

clf=clf.fit(X_train, y_train)  
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)                             

0.81118881118881114

In [24]:
## STACKING

clf = tree.DecisionTreeClassifier(max_leaf_nodes=25)
clf.fit(X_train,y_train)
prediction=clf.predict(X_train)

clf1 = GradientBoostingClassifier(n_estimators=100, learning_rate=.1,max_depth=1, random_state=0)
clf1.fit(X_train,y_train)
prediction1=clf1.predict(X_train)

clf2 = RandomForestClassifier(n_estimators=100)
clf2.fit(X_train,y_train)
prediction2=clf2.predict(X_train)

dataL2=pd.DataFrame(prediction,columns=['p1'])

dataL2=pd.concat([dataL2 ,pd.DataFrame(prediction1,columns=['p2']), pd.DataFrame(prediction2,columns=['p3'])],axis=1)

dataL2.head()

,p1,p2,p3
0,1,0,1
1,0,0,0
2,1,1,1
3,1,0,0
4,0,0,1


In [25]:
from sklearn.linear_model import LogisticRegression

L2X=dataL2.values
L2y=y_train

finalClassifier=LogisticRegression()
finalClassifier.fit(L2X,L2y)

# Test Data Generation
testPrediction=clf.predict(X_test)
testPrediction1=clf1.predict(X_test)
testPrediction2=clf2.predict(X_test)

testL2=pd.DataFrame(testPrediction,columns=['p1'])

testL2=pd.concat([testL2 ,pd.DataFrame(testPrediction1,columns=['p2']), pd.DataFrame(testPrediction2,columns=['p3'])],axis=1)

finalPred=finalClassifier.predict(testL2.values)

accuracy_score(y_test, finalPred) 

0.80419580419580416

In [30]:
import sklearn
sklearn.__version__

'0.18.1'